In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16,MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os

In [3]:
import tensorflow as tf
tf.__version__

'2.2.0'

In [0]:
from skimage import io

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
ls 'drive/My Drive/facemask'

withmask/  withoutmask/


In [0]:
# initial learning rate, number of epochs to train for and batch size
INIT_LR = 1e-3
EPOCHS = 40
BS = 8
# taking list of images in our directory

imagePaths = list(paths.list_images('drive/My Drive/facemask'))
data = []
labels = []

In [0]:
# loop over the image paths
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]

	# Reading the images
	image = io.imread(imagePath)
	# swapping the colour channels
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  # Resizing the image to 224, 224
	image = cv2.resize(image, (224, 224))

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)


In [0]:
# convert the data and labels to NumPy arrays while scaling the pixel 
data = np.array(data) / 255.0
labels = np.array(labels)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

# Training and text data split

(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)

In [0]:
# Data augmentation to have a varaince in the data
trainAug = ImageDataGenerator(
	rotation_range=20,
  width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.1,
	fill_mode="nearest")

In [11]:
## Using VGG16 architecture and imagenet weights
baseModel = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

58892288/58889256 [==============================] - 0s 0us/step


In [0]:
## Head mdel
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

## Head and base model
model = Model(inputs=baseModel.input, outputs=headModel)
for layer in baseModel.layers:
	layer.trainable = False

In [13]:
# compile our model
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# training head network
H = model.fit_generator(
	trainAug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/40
25/25 [==============================] - 7s 275ms/step - loss: 0.6222 - accuracy: 0.6250 - val_loss: 0.3301 - val_accuracy: 0.9804
Epoch 2/40
25/25 [==============================] - 4s 164ms/step - loss: 0.3430 - accuracy: 0.8950 - val_loss: 0.1996 - val_accuracy: 0.9804
Epoch 3/40
25/25 [==============================] - 4s 163ms/step - loss: 0.2286 - accuracy: 0.9450 - val_loss: 0.1406 - val_accuracy: 0.9804
Epoch 4/40
25/25 [==============================] - 4s 164ms/step - loss: 0.1639 - accuracy: 0.9700 - val_loss: 0.1135 - val_accuracy: 0.9608
Epoch 5/40
25/25 [==============================] - 4s 164ms/step - loss: 0.1462 - accuracy: 0.9650 - val_loss: 0.1171 - val_accuracy: 0.9412
Epoch 6/40
25/25 [==============================] - 4s 164ms/step - loss: 0.1154 - accuracy: 0.9700 - val_loss: 0.1200 - val_accuracy: 0.9412
Epoch 7/40
25/25 [==============================] - 4s 163ms/step - loss

In [0]:
model.save('drive/My Drive/mask_model.h5') 